# Modes and Separability for the Square Core fiber

Here we find the modes and check to see if they are product functions (ie the equal products of functions only of x and y individually).

In [ ]:
import ngsolve as ng
import numpy as np
import matplotlib.pyplot as plt
from ngsolve.webgui import Draw
from fiberamp.fiber.microstruct.pbg import SquareFiber

In [ ]:
A = SquareFiber(refine=1, curve=5, corner_maxh=.01)

In [ ]:
Draw(A.V, A.mesh)

In [ ]:
z, yr, _, _, _, _ = A.leakymode(2, ctr=3.3j, rad = .1, alpha=10, npts=4, nspan=2, niterations=10,nrestarts=0 )

In [ ]:
for f in yr:
    Draw(1e2*f, A.mesh)


# Plot cross sections of core 

In [ ]:
S = np.linspace(-A.Rsqr_in, A.Rsqr_in, 301)

In [ ]:
x0, y0 = 0,0
f = yr[0]
xs, ys = [A.mesh(s, y0) for s in S], [A.mesh(x0, s) for s in S]
fxs = np.array([f(xpt) for xpt in xs])
fys = np.array([f(ypt) for ypt in ys])

In [ ]:
%matplotlib inline
plt.plot(S, fxs.real)
# plt.plot(S, fxs.imag)



In [ ]:
plt.plot(S, fys.real)
# plt.plot(S, fys.imag)

That's extremely symmetric

# Let's look at the z component of the vector mode

In [ ]:
betas, zs, E, phi, _ = A.leakyvecmodes(ctr=-10.889, rad=.1, p=2, npts=4, nspan=3, 
                                       niterations=20, nrestarts=0, stop_tol=1e-9)

In [ ]:
for p in E:
    Draw(p.real, A.mesh)

# Cross section

In [ ]:
x0, y0 = .2,0
F = phi[0]
xs, ys = [A.mesh(s, y0) for s in S], [A.mesh(x0, s) for s in S]
Fxs = np.array([F(x) for x in xs], dtype=complex)
Fys = np.array([F(y) for y in ys], dtype=complex)

In [ ]:
%matplotlib inline
plt.plot(S, Fxs.real)
plt.plot(S, Fxs.imag)


In [ ]:
plt.plot(S, Fys.real)
plt.plot(S, Fys.imag)

# Create product function

To see if the above really are product functions, we should be able to recreate them using the two cross sections we have

In [ ]:
from mpl_toolkits import mplot3d

In [ ]:
u_0 = F(A.mesh(x0,y0))

In [ ]:
u_0

In [ ]:
X, Y = np.meshgrid(S,S)
Z = np.outer(Fxs, Fys)

In [ ]:
%matplotlib notebook
fig = plt.figure(figsize=(12,8))
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, cmap='viridis');
# ax.plot_surface(X, Y, Z.imag, cmap='viridis')

In [ ]:
Z_true = np.zeros_like(X, dtype=complex)

In [ ]:
for i,x in enumerate(X):
    for j, y in enumerate(Y):
        Z_true[i,j] = F(A.mesh(x[i],y[j]))

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z_true.real, cmap='viridis')
ax.plot_surface(X, Y, Z.real, cmap='turbo')

In [ ]:
%matplotlib notebook
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, (Z-Z_true).real, cmap='viridis')

# Does seem likely that this is a product function

We are having troubles getting it in the corners, but this isn't necessarily because it's not a product but because corners are hard to capture using finite elements.

It's also good that the imaginary part of the ratio of the two is near machine zero.

# Improve the finite element approach

If we made our outer boundary square, we expect that we know the radial derivative, it would be related to exponential decay and our eigenvalue (in some way).  We could try to implement that, or also we could just do Neumann conditions on a square outer boundary far from the core.  We could also use CSG 2D and up the precision at the corners using point info and decreasing the maxh there.  

All worth doing, but also it seems likely that we can go ahead theoretically and try to figure out the product function theory.  We'd have to line up regions etc.  We could try to get a sense of what the constants of separation are on each region using the numerics from this notebook (especially after implementing some of the above fixes).